[View in Colaboratory](https://colab.research.google.com/github/NgoDinh/Data-science/blob/master/odoo_bc_khach_hang.ipynb)

In [0]:
# -*- coding: utf-8 -*-
##############################################################################
#
#    OpenERP, Open Source Management Solution
#
##############################################################################
import time
import os
import base64
import pandas as pd
import psycopg2 as pg
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from sqlalchemy import create_engine
from openerp.osv import fields, osv
from openerp.tools.translate import _
from openerp.tools import DEFAULT_SERVER_DATE_FORMAT, DEFAULT_SERVER_DATETIME_FORMAT
from dateutil import parser

class report_online_performance(osv.osv_memory):
    
    _name = "report.online.performance"
    _columns = {
        'date_start': fields.date('Date From', required=True),
        'date_end': fields.date('Date To', required=True),
        'file_output':fields.binary(string="File Output",invisible=True, readonly=True, help='Output file in xlsx format'),
        'file_name':fields.char(string='File Name', invisible=True)
    }
    _defaults = {
         'date_start': time.strftime('%Y-%m-01'),
         'date_end': lambda *a: str(datetime.now() + relativedelta(months=+1, day=1, days=-1))[:10],
    }
    
#     def print_report(self, cr, uid, ids, context=None):
#         if context is None:
#             context = {}
#         datas = {'ids': context.get('active_ids', [])}
#         res = self.read(cr, uid, ids, ['date_start', 'date_end'], context=context)
#         res = res and res[0] or {}
#         datas['form'] = res
#         if res.get('id',False):
#             datas['ids']=[res['id']]
#         return {'type': 'ir.actions.report.xml', 'report_name': 'bc_doanh_thu_theo_location' , 'datas': datas}
    
    def print_excel(self, cr, uid, ids, context=None):

       
        #set date to query_data
        #============================================================================================#        
        datas = {'ids': context.get('active_ids', [])}
        date_start = self.read(cr, uid, ids, ['date_start'], context=context)
        date_start = date_start[0]['date_start']+' 00:00:00'
        date_end   = self.read(cr, uid, ids, ['date_end'], context=context)
        date_end   = date_end[0]['date_end']+' 23:59:59'
        out_path = '/tmp/bc_khach_hang_%s.xlsx'%(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

        #Run report
        #============================================================================================# 
        def get_data(date_start, date_end, channel = 'online'):

            if channel == 'online':
                # get data from online customers
                get_data_query = '''
                SELECT Count(tb.id)::int as slkh, tb.sl::int as solan , round(sum(tb.TongDT),0) as doanhthu, round(sum(tb.TongDT)/Count(tb.id),0) as tbinh
                FROM (
                    SELECT DISTINCT r.id, count(s.id) as SL, sum(amount_total) as TongDT
                    FROM res_partner r INNER JOIN sale_order s ON r.id = s.partner_id
                    WHERE  s.state NOT IN ('draft','cancel') AND s.date_order between '%s' AND '%s' 
                        and s.section_id in ( select id from crm_case_section where name not like 'Direct%%' or s.section_id is NULL)
                        and s.name NOT LIKE '%s-%s'
                    GROUP BY r.id
                    ORDER BY SL DESC ) tb
                GROUP BY tb.sl
                ORDER BY tb.sl  DESC
                '''%(date_start, date_end,'%','%')
            else:
                # get data from offline customers
                get_data_query = '''
                SELECT Count(tb.id)::int as slkh, tb.sl::int as solan , round(sum(tb.TongDT),0) as doanhthu, round(sum(tb.TongDT)/Count(tb.id),0) as tbinh
                FROM (
                    SELECT DISTINCT r.id, count(distinct s.id) as SL, sum(l.price_unit*l.qty) as TongDT
                    FROM res_partner r 
                        INNER JOIN pos_order s ON r.id = s.partner_id
                        LEFT JOIN pos_order_line l ON l.order_id = s.id
                    WHERE  s.state IN ('paid', 'done', 'invoiced') AND s.date_order between '%s' AND '%s' 
                    GROUP BY r.id
                    ORDER BY SL DESC ) tb
                GROUP BY tb.sl
                ORDER BY tb.sl  DESC
                '''%(date_start, date_end)

            engine = pg.connect('postgresql://odoo:odoo@***:***/***')
            df = pd.read_sql(get_data_query,engine)
            df = df[['solan', 'slkh', 'doanhthu', 'tbinh']]

            return df
        
        def run_report(date_start, date_end):

            df = get_data(date_start, date_end, 'online')
            df_1 = get_data(date_start, date_end, 'offline')

            #create excel file
            #out_path = r'C:\Users\admin\Downloads\final_report.xlsx'
            writer = pd.ExcelWriter(out_path , engine='xlsxwriter')

            df_tmp = df.copy()
            df_tmp.columns = ['Số lần mua hàng', 'Số lượng khách hàng', 'Tổng doanh thu', 'Doanh thu trung bình']
            df_tmp.to_excel(writer,sheet_name = 'online', startrow=7, startcol=4, index=False)

            df_tmp = df_1.copy()
            df_tmp.columns = ['Số lần mua hàng', 'Số lượng khách hàng', 'Tổng doanh thu', 'Doanh thu trung bình']
            df_tmp.to_excel(writer,sheet_name = 'offline', startrow=7, startcol=4, index=False)

            workbook  = writer.book
            online = writer.sheets['online']
            offline = writer.sheets['offline']

            #format for workbook
            header_format = workbook.add_format({'bold': True, 'font_color': 'black', 'bg_color': '#B7DEE8',  'border':1})
            number_format = workbook.add_format({'border':1, 'align': 'center', 'num_format':'#,##0.00'})
            date_format = workbook.add_format({'border':1, 'align': 'center', 'bold':True})
            big_size = workbook.add_format({'bold': True, 'font_size':15})

            def write_excel(df, worksheet, date_start, date_end):
                ''' append some index to report'''

                len_df = len(df)
                date_start = date_start
                date_end = date_end
                gap = (parser.parse(date_end) - parser.parse(date_start)).days

                #write hearder of report
                worksheet.write(0, 5, 'BÁO CÁO KHÁCH HÀNG %s'%(worksheet.get_name().upper()), big_size)
                worksheet.write(2, 4, 'Start Date',header_format)
                worksheet.write(2, 5, date_start, date_format)
                worksheet.write(3, 4, 'Start End', header_format)
                worksheet.write(3, 5, date_end, date_format)
                worksheet.write(4, 4, 'Total Days', header_format)
                worksheet.write(4, 5, gap, date_format)

                #repeat customer rate
                c_more_than_1 = sum(df[df['solan']>1]['slkh'])
                unique_c = sum(df['slkh'])
                repeat_c_rate = c_more_than_1/unique_c

                worksheet.write(12 + len(df), 4, "Repeat Customer Rate", header_format)
                worksheet.write(13 + len(df), 4, "Customers That Have Purchased More Than Once", header_format)
                worksheet.write(13 + len(df), 5, c_more_than_1, number_format)
                worksheet.write(14 + len(df), 4, "Unique Customers", header_format)
                worksheet.write(14 + len(df), 5, unique_c, number_format)
                worksheet.write(15 + len(df), 4, "Repeat Customer Rate", header_format)
                worksheet.write(15 + len(df), 5, repeat_c_rate, number_format)

                #purchase frequency
                total_order = sum(df['solan']*df['slkh'])
                purchase_frequency = total_order / unique_c
                time_between_purchase = gap/purchase_frequency

                worksheet.write(18 + len(df), 4, "Purchase Frequency", header_format)
                worksheet.write(19 + len(df), 4, "Total Orders", header_format)
                worksheet.write(19 + len(df), 5, total_order, number_format)
                worksheet.write(20 + len(df), 4, "Unique Customers", header_format)
                worksheet.write(20 + len(df), 5, unique_c, number_format)
                worksheet.write(21 + len(df), 4, "Purchase Frequency", header_format)
                worksheet.write(21 + len(df), 5, purchase_frequency, number_format)
                worksheet.write(22 + len(df), 4, "Time Between Purchases (TBP) (đv: ngày)", header_format)
                worksheet.write(22 + len(df), 5, time_between_purchase, number_format)


                #loyal customer rate
                c_emore_than_4 = sum(df[df['solan']>=4]['slkh'])
                loyal_c_rate = c_emore_than_4 / unique_c

                worksheet.write(24 + len(df), 4, "Loyal Customer Rate", header_format)
                worksheet.write(25 + len(df), 4, "Customers That Have Purchased 4+ Times", header_format)
                worksheet.write(25 + len(df), 5, c_emore_than_4, number_format)
                worksheet.write(26 + len(df), 4, "Unique Customers", header_format)
                worksheet.write(26 + len(df), 5, unique_c, number_format)
                worksheet.write(27 + len(df), 4, "Loyal Customer Rate", header_format)
                worksheet.write(27 + len(df), 5, loyal_c_rate, number_format)

                #average order value
                total_rev = sum(df['doanhthu'])
                AOV = total_rev / total_order

                worksheet.write(12 + len(df), 7, "Average Order Value (AOV)", header_format)
                worksheet.write(13 + len(df), 7, "Total Revenue", header_format)
                worksheet.write(13 + len(df), 8, total_rev, number_format)
                worksheet.write(14 + len(df), 7, "Orders Taken", header_format)
                worksheet.write(14 + len(df), 8, total_order, number_format)
                worksheet.write(15 + len(df), 7, "Average Order Value (AOV)", header_format)
                worksheet.write(15 + len(df), 8, AOV, number_format)

                #customer lifetime value
                eight_months = purchase_frequency * AOV * (2/3)
                one_year = purchase_frequency * AOV
                two_years = purchase_frequency * AOV * 2
                three_years = purchase_frequency * AOV * 3

                worksheet.write(18 + len(df), 7, 'Customer Lifetime Value (CLV) (đ)', header_format)
                worksheet.write(19 + len(df), 7, '< 1 Year ~ 8 Month', header_format)
                worksheet.write(19 + len(df), 8, eight_months, number_format)
                worksheet.write(20 + len(df), 7, '1 Year', header_format)
                worksheet.write(20 + len(df), 8, one_year, number_format)
                worksheet.write(21 + len(df), 7, '2 Years', header_format)
                worksheet.write(21 + len(df), 8, two_years, number_format)
                worksheet.write(22 + len(df), 7, '3 Years', header_format)
                worksheet.write(22 + len(df), 8, three_years, number_format)


                #write total row
                worksheet.write(8 + len(df), 4, "Tổng Cộng", header_format)
                worksheet.write(8 + len(df), 5, unique_c, number_format)
                worksheet.write(8 + len(df), 6, total_rev, number_format)
                worksheet.write(8 + len(df), 7, total_rev / unique_c, number_format)

                worksheet.set_column('A:C', 1)
                worksheet.set_column('E:E', 47)
                worksheet.set_column('F:I', 32)
                worksheet.hide_gridlines(2)

                for row in range(len_df):
                    for col in range(4):
                        worksheet.write(8 + row, 4 +col, df.iloc[row][col], number_format)

            write_excel(df, online, date_start, date_end)
            write_excel(df_1, offline, date_start, date_end)

            writer.save()    
        
        
        #Encode final file to binary and download it.
        #============================================================================================#
        
        PREVIEW_PATH = out_path
        encoded_string = ""     
        with open(PREVIEW_PATH, "r") as image_file:
            encoded_string = base64.encodestring(image_file.read())#.b64encode
        for order in self.browse(cr, uid, ids[0]):
            order.file_name = '%s.xlsx'%(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            order.file_output = encoded_string
            data_id =  order.id
         
        return {
            'type': 'ir.actions.act_window',
            'res_model': 'report.online.performance',
            'view_mode': 'form',
            'view_type': 'form',
            'res_id': data_id,
            'views': [(False, 'form')],
            'target': 'new',
        }

# vim:expandtab:smartindent:tabstop=4:softtabstop=4:shiftwidth=4:
